In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import the CRUD module, AnimalShelter method via Module4_AAC
from Module4_AAC import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Updated username and password and CRUD Python module name
# Updated port, database as DB and collection as COL
username = "aacuser"
password = "CS340Password"
HOST = 'nv-desktop-services.apporto.com'
PORT = 32370
DB = 'AAC'
COL = 'animals'

db = AnimalShelter(username, password, HOST, PORT, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################

# Updated Jupyter Dash name to 'GraziosoSalvare'
app = JupyterDash('GraziosoSalvare') 

# Updated to Grazioso Salvare’s logo
image_filename = 'Grazioso-Salvare-Logo.png' # Image Renamed & Replaced
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Added Grazioso Salvare's logo centered above 'Animal Database'
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
    width=250, height=250)),
    # Titled the dashboard as Animal Database
    html.Center(html.B(html.H1('Animal Database'))),
    #Unique Identifier, centered, and significant to programmer
    html.Center(html.B(html.H6('CS-340 VJM717'))),
    html.Hr(),
    # Interactive Filtering Options - Dropdown Menu Implemented
    # Dropdown options include Water Rescue, Maountain or Wilderness Rescue,
    # Disaster or Individual Tracking, and Reset
    html.Div(
        children=[
            html.Label("Rescue Type"),
            dcc.Dropdown(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Forest'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                    {'label': 'Reset', 'value': 'Reset'},
                ],
                value = 'Reset',
        
            )
        ]),

    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
    # Features for Interactive Data Table Implemented
                         editable = False,
                         filter_action = "native",
                         sort_action = "native", 
                         sort_mode = "multi", 
                         column_selectable = "single", 
                         row_selectable = "single", 
                         row_deletable = False, 
                         selected_columns = [],
                         selected_rows = [0],
                         page_action = "native", 
                         page_current = 0, 
                         page_size = 10, 

                        ),
    html.Br(),
    html.Hr(),
    
# Chart and Geolocation Chart 
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

#############################################
# 
# Per Requirements, filtering options include
# desired animal type, breeds, sex upon outcome,
# and age upon outcome to meet the client's 
# requirements. 
#
#############################################

# callback for interactive filtering options
@app.callback(
    Output('datatable-id','data'),
    [Input('filter-type', 'value')])

def update_dashboard(filter_type):
    
    df = None
    
    # filtering for water rescue type
    if filter_type == "Water": 
        df = pd.DataFrame.from_records(db.read( 
                {"$and":[
                    {'animal_type': "Dog"},
                    {'sex_upon_outcome': "Intact Female"},
                    {"age_upon_outcome_in_weeks": {"$gte": 26}},
                    {"age_upon_outcome_in_weeks": {"$gte": 156}},
                    {"$or":[
                        {'breed': "Labrador Retriever Mix"},
                        {'breed': "Chesapeake Bay Retriever"},
                        {'breed': "Newfoundland"},
                    ]}
                ]}))
    
    # filtering for mountain or wilderness rescue type
    elif filter_type == "Forest": 
        df = pd.DataFrame.from_records(db.read(
                {"$and":[
                    {'animal_type': "Dog"},
                    {'sex_upon_outcome': "Intact Male"},
                    {"age_upon_outcome_in_weeks": {"$gte": 26}},
                    {"age_upon_outcome_in_weeks": {"$gte": 156}},
                    {"$or":[
                        {'breed': "German Shepherd"},
                        {'breed': "Alaskan Malamute"},
                        {'breed': "Old English Sheepdog"},
                        {'breed': "Siberian Husky"},
                        {'breed': "Rottweiler"},
                    ]}
                ]}))
        
    # filtering for disaster or individual tracking type
    elif filter_type == "Disaster": 
        df = pd.DataFrame.from_records(db.read(
                {"$and":[
                    {'animal_type': "Dog"},
                    {'sex_upon_outcome': "Intact Male"},
                    {"age_upon_outcome_in_weeks": {"$gte": 20}},
                    {"age_upon_outcome_in_weeks": {"$gte": 300}},
                    {"$or":[
                        {'breed': "Doberman Pinscher"},
                        {'breed': "German Shepherd"},
                        {'breed': "Goldern Retriever"},
                        {'breed': "Bloodhound"},
                        {'breed': "Rottweiler"},
                    ]}
                ]}))
    
    # reset database to all animals otherwise
    else:
        df = pd.DataFrame.from_records(db.read({}))
        
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

# callback for graph implementation
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    
    if viewData is None:
        return
    
    # implementation of pie chart for preferred animal breeds
    df = pd.DataFrame.from_records(viewData)
    return[
        dcc.Graph(
            figure=px.pie(df,names = 'breed', title = 'Preferred Breed')
        )
    ]
    
# callback to highlight a cell on the data table when selected
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# callback to update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '750px', 'height': '650px'}, 
               center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], 
                      children=[
                          dl.Tooltip(dff.iloc[row,4]),
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:13282/
